# Câu 1:

In [16]:
import numpy as np
import pandas as pd
import math
import copy

In [10]:
dataset = pd.read_csv('/content/drive/MyDrive/HK3_P/AI/AI_Baitap_Tuan2/PlayTennis.csv')
X = dataset.iloc[:, 1:].values
print(X)
attribute = ['outlook', 'temp', 'humidity', 'wind']

[['Sunny' 'Hot' 'High' 'Weak' 'No']
 ['Sunny' 'Hot' 'High' 'Strong' 'No']
 ['Overcast' 'Hot' 'High' 'Weak' 'Yes']
 ['Rain' 'Mild' 'High' 'Weak' 'Yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'No']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'Yes']
 ['Sunny' 'Mild' 'High' 'Weak' 'No']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'Yes']
 ['Sunny' 'Mild' 'Normal' 'Strong' 'Yes']
 ['Overcast' 'Mild' 'High' 'Strong' 'Yes']
 ['Overcast' 'Hot' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Mild' 'High' 'Strong' 'No']]


In [17]:
class Node(object):
    def __init__(self):
        self.value = None
        self.decision = None
        self.childs = None
 
def findEntropy(data, rows):
    yes = 0
    no = 0
    ans = -1
    idx = len(data[0]) - 1
    entropy = 0
    for i in rows:
        if data[i][idx] == 'Yes':
            yes = yes + 1
        else:
            no = no + 1
 
    x = yes/(yes+no)
    y = no/(yes+no)
    if x != 0 and y != 0:
        entropy = -1 * (x*math.log2(x) + y*math.log2(y))
    if x == 1:
        ans = 1
    if y == 1:
        ans = 0
    return entropy, ans
 
def findMaxGain(data, rows, columns):
    maxGain = 0
    retidx = -1
    entropy, ans = findEntropy(data, rows)
    if entropy == 0:
        """if ans == 1:
            print("Yes")
        else:
            print("No")"""
        return maxGain, retidx, ans
 
    for j in columns:
        mydict = {}
        idx = j
        for i in rows:
            key = data[i][idx]
            if key not in mydict:
                mydict[key] = 1
            else:
                mydict[key] = mydict[key] + 1
        gain = entropy
 
        # print(mydict)
        for key in mydict:
            yes = 0
            no = 0
            for k in rows:
                if data[k][j] == key:
                    if data[k][-1] == 'Yes':
                        yes = yes + 1
                    else:
                        no = no + 1
            # print(yes, no)
            x = yes/(yes+no)
            y = no/(yes+no)
            # print(x, y)
            if x != 0 and y != 0:
                gain += (mydict[key] * (x*math.log2(x) + y*math.log2(y)))/14
        # print(gain)
        if gain > maxGain:
            # print("hello")
            maxGain = gain
            retidx = j
 
    return maxGain, retidx, ans
 
def buildTree(data, rows, columns):
    maxGain, idx, ans = findMaxGain(X, rows, columns)
    root = Node()
    root.childs = []
    # print(maxGain
    #
    # )
    if maxGain == 0:
        if ans == 1:
            root.value = 'Yes'
        else:
            root.value = 'No'
        return root
 
    root.value = attribute[idx]
    mydict = {}
    for i in rows:
        key = data[i][idx]
        if key not in mydict:
            mydict[key] = 1
        else:
            mydict[key] += 1
            
    newcolumns = copy.deepcopy(columns)
    newcolumns.remove(idx)
    for key in mydict:
        newrows = []
        for i in rows:
            if data[i][idx] == key:
                newrows.append(i)
        # print(newrows)
        temp = buildTree(data, newrows, newcolumns)
        temp.decision = key
        root.childs.append(temp)
    return root
 
def traverse(root):
    print(root.decision)
    print(root.value)
 
    n = len(root.childs)
    if n > 0:
        for i in range(0, n):
            traverse(root.childs[i])
 
def calculate():
    rows = [i for i in range(0, 14)]
    columns = [i for i in range(0, 4)]
    root = buildTree(X, rows, columns)
    root.decision = 'Start'
    traverse(root)

calculate()

Start
outlook
Sunny
humidity
High
No
Normal
Yes
Overcast
Yes
Rain
wind
Weak
Yes
Strong
No


## Source

Import thư viện và tiến hành đọc .csv

In [19]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
data=pd.read_csv("/content/drive/MyDrive/HK3_P/AI/AI_Baitap_Tuan2/PlayTennis.csv")

Xác định entropy mục tiêu

In [20]:
#Entropy cho thuộc tính mục tiêu
def find_entropy(df):
  target = df.keys()[-1] #(playGolf)
  entropy = 0
  values = df[target].unique() #yes,no
  for value in values:
    fraction = df[target].value_counts()[value]/len (df[target])
    entropy += -fraction * np.log2(fraction)
  return entropy

Tính Entropy thuộc tính

In [21]:
def find_entropy_atrribute(df,attribute):
  target = df.key()[-1]
  target_variables = df[target].unique()
  #unique values in target playGolf attribute (Yes,No)
  variables = df[attribute].unique() #Sunny, Overcast, Rainy
  #attribute entropy
  #Variables=[sunny, sunny....5, overcast1....overcast4, Rainy1...Ra5 ]
  entropy2=0
  for variable in variables:
    entropy=0
    for target_variable in target_variables:
      num=len(df[attribute][df[attribute]==variable][df[target]==target_variable])
      den=len(df[attribute][df[attribute]==variable])
      fraction = num/(den+eps)
      entropy+= -fraction*np.log2(fraction+eps)
    fraction=den/len(df)
    entropy2+=-fraction*entropy
    return abs(entropy2)

Xác định IG

In [22]:
def bestClassifier(df):
  IG = []
  for key in df.keys()[:-1]:
      IG.append(find_entropy(df) -find_entropy_atrribute(df, key))
  return df.keys()[:-1][np.argmax(IG)]

Xây dựng cây quyết định

In [34]:
def get_subtable(df, node, value):
  return df[df[node]==value].reset_index(drop=True)
def ID3split(df, tree=None):
  target = df.keys()[-1]
  node = bestClassifier(df)
  attributeValues = np.unique(df[node])
  if tree is None:
    tree = {}
    tree =[node] = {}
  for value in attributeValues:
    subtable = get_subtable(df, node, value)
    targetValues, counts = np.unique(subtable[target], return_counts=True)
    if len(counts) == 1:
      tree[node][value] = targetValues[0]
    else:
      tree[node][value] = ID3split(subtable)
  return tree

# Câu 2: Vẽ Decision Tree

In [35]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from IPython.display import Image
from io import StringIO
import pydotplus
from sklearn import preprocessing, tree

In [37]:
def plot_decision_tree(clf, feature_name, target_name):
  dot_data = StringIO()
  tree.export_graphviz(clf, out_file=dot_data,
                       feature_names=feature_name,
                       class_names=target_name,
                       filled=True, rounded=True,
                       special_characters=True)
  graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
  return Image(graph.create_png())